In [57]:
# Time Series methods - actually trying to use the direct time data or a processed version of it as input to models
import mne
import numpy as np
import pandas as pd
import sklearn
from sklearn import svm 
from sklearn.svm import SVC 
from sklearn.svm import NuSVC
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import VotingClassifier as VC
from sklearn.neighbors import KNeighborsClassifier as sknn
from sklearn.metrics import confusion_matrix as cm
from sklearn.ensemble import StackingClassifier as SC

from mne.externals.pymatreader import read_mat

from pyriemann.utils.distance import distance_riemann
from pyriemann.classification import MDM
from pyriemann.classification import KNearestNeighbor as knn
from pyriemann.classification import TSclassifier as tsc
from pyriemann.classification import FgMDM
from pyriemann.estimation import Covariances
from pyriemann.estimation import (XdawnCovariances, HankelCovariances,
                                  CospCovariances, ERPCovariances)
from pyriemann.spatialfilters import Xdawn, CSP
from pyriemann.tangentspace import TangentSpace


import xgboost as xgb

import matplotlib.pyplot as plt
%matplotlib qt

import joblib
import os
import time

mne.set_log_level('ERROR')

In [2]:
# Different inputs for the TSclassifier, maybe including the other Coherence methods
from sklearn.base import BaseEstimator, TransformerMixin

class EpochsVectorizer(BaseEstimator, TransformerMixin):
    """Vectorize epochs."""

    def __init__(self):
        """Init."""

    def fit(self, X, y):
        return self

    def transform(self, X):
        X2 = np.array([x.flatten() for x in X])
        return X2


class CospBoostingClassifier(BaseEstimator, TransformerMixin):
    """Cospectral matrice bagging."""

    def __init__(self, baseclf):
        """Init."""
        self.baseclf = baseclf

    def fit(self, X, y):
        self.clfs_ = []
        for i in range(X.shape[-1]):
            clf = deepcopy(self.baseclf)
            self.clfs_.append(clf.fit(X[:, :, :, i], y))
        return self

    def predict_proba(self, X):
        proba = []
        for i in range(X.shape[-1]):
            proba.append(self.clfs_[i].predict_proba(X[:, :, :, i]))
        proba = np.mean(proba, axis=0)
        return proba

    def transform(self, X):
        proba = []
        for i in range(X.shape[-1]):
            proba.append(self.clfs_[i].predict_proba(X[:, :, :, i]))
        proba = np.concatenate(proba, 1)
        return proba

In [2]:
%run funcs.ipynb

In [3]:
# Extract the X_train - 1st subject, 1st session data
# get the epochs data of first subject first trial
cwd = os.getcwd()
data_path = cwd

sub_n= 0
session_n = 0

# epochs = {}
epochs_data = []
labels = []
i = 0
chans = []
diff = ['MATBeasy', 'MATBmed', 'MATBdiff']
for lab_idx, level in enumerate(diff):
    sub = 'P{0:02d}'.format(sub_n+1)
    sess = f'S{session_n+1}'
    path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{session_n+1}_{level}.set'
    # Read the epoched data with MNE
    epochs = mne.io.read_epochs_eeglab(path, verbose=False)
    if i == 0:
        chans = epochs.ch_names
        i = i+1
    # You could add some pre-processing here with MNE
    tmp = epochs.get_data()
    epochs_data.extend(tmp)
    labels.extend([lab_idx]*len(tmp))

X1 = np.array(epochs_data)
y1 = np.array(labels)
print(X1.shape)
print(y1.shape)
print(chans) 
# Just use this as a reference - the features will generated as range(n_channels) 
# - Should be in this order


(447, 61, 500)
(447,)
['Fp1', 'Fz', 'F3', 'F7', 'FT9', 'FC5', 'FC1', 'C3', 'T7', 'CP5', 'CP1', 'Pz', 'P3', 'P7', 'O1', 'Oz', 'O2', 'P4', 'P8', 'CP6', 'CP2', 'FCz', 'C4', 'T8', 'FT8', 'FC6', 'FC2', 'F4', 'F8', 'Fp2', 'AF7', 'AF3', 'AFz', 'F1', 'F5', 'FT7', 'FC3', 'C1', 'C5', 'TP7', 'CP3', 'P1', 'P5', 'PO7', 'PO3', 'POz', 'PO4', 'PO8', 'P6', 'P2', 'CPz', 'CP4', 'TP8', 'C6', 'C2', 'FC4', 'FT10', 'F6', 'AF8', 'AF4', 'F2']


In [4]:
# first subject second session
cwd = os.getcwd()
data_path = cwd

sub_n= 0
session_n = 1

# epochs1 = {}
epochs_data1 = []
labels1 = []
diff = ['MATBeasy', 'MATBmed', 'MATBdiff']
for lab_idx, level in enumerate(diff):
    sub = 'P{0:02d}'.format(sub_n+1)
    sess = f'S{session_n+1}'
    path = os.path.join(os.path.join(data_path, sub), sess) + f'/eeg/alldata_sbj{str(sub_n+1).zfill(2)}_sess{session_n+1}_{level}.set'
    # Read the epoched data with MNE
    epochs = mne.io.read_epochs_eeglab(path, verbose=False)
    tmp = epochs.get_data()
    epochs_data1.extend(tmp)
    labels1.extend([lab_idx]*len(tmp))
# 
X2 = np.array(epochs_data1)
y2 = np.array(labels1)
print(X2.shape)
print(y2.shape)

(447, 61, 500)
(447,)


In [5]:
# direct load and vectorize and give to LDA
# vectorize
vx1 = X1.reshape(X1.shape[0],-1)
vx2 = X2.reshape(X2.shape[0],-1)

scaler = StandardScaler()
sx1 = scaler.fit_transform(vx1)
sx2 = scaler.transform(vx2)

print(f'Average feature-wise mean: {sx1.mean():0.5f}')
print('Average feature-wise standard deviation: '
      f'{sx1.std(axis=0).mean():0.5f}')
# np.mean(sx1,axis=0)

Average feature-wise mean: 0.00000
Average feature-wise standard deviation: 1.00000


In [14]:
log_reg = LogisticRegression(random_state=87,max_iter = 1000).fit(sx1, y1)
test_acc = log_reg.score(sx2, y2)

print(f'\nAccuracy of vect + log reg: {test_acc * 100:0.2f}%\n')


Accuracy of vect + log reg: 38.03%



In [15]:
log_reg = LogisticRegression(random_state=87,max_iter = 1000).fit(sx2, y2)
test_acc = log_reg.score(sx1, y1)

print(f'\nAccuracy of vect + log reg: {test_acc * 100:0.2f}%\n')


Accuracy of vect + log reg: 36.02%



In [19]:
# Riemannian Geometry method
# remove DC offset - Already done in the dataset
# print(X1[0,0,:])
# nx1 = X1 - X1.mean(axis=-1, keepdims=True)
# print(nx1[0,0,:])

# create covariance matrix - transpose the CxT to TxC and multiply the matrices
covs_train = (X1 @ X1.transpose(0, 2, 1)) / (
    X1.shape[-1] - 1)
print(f'Training set covariances have shape: {covs_train.shape}')

Training set covariances have shape: (447, 61, 61)


In [27]:
# Viz the cov mat
ind = 30
# ch_names = [f'ch{i}' for i in range(1, 5)]
max_ampl = np.abs(covs_train[ind]).max()  # find max abs(value) for plotting

fig, ax = plt.subplots()
im = ax.imshow(covs_train[ind], cmap='bwr', vmin=-max_ampl, vmax=max_ampl)
ax.set_yticks(range(61))
ax.set_yticklabels(chans)
ax.set_xticks(range(61))
ax.set_xticklabels(chans)
fig.colorbar(im, ax=ax, fraction=0.05)
ax.set_title('A covariance matrix\nin our dataset')
fig.tight_layout()

In [44]:
# Min distance to Mean - compare the covariance matrix of that window to the
# average covariance matrix for each class :math:`\Sigma_{class~k}` - whichever one
# has the smallest distance is then used as label prediction

class0_cov = covs_train[y1 == 0].mean(axis=0)
class1_cov = covs_train[y1 == 1].mean(axis=0)
class2_cov = covs_train[y1 == 2].mean(axis=0)

eucl_dist0 = np.sum(np.abs(covs_train[0] - class0_cov))
eucl_dist1 = np.sum(np.abs(covs_train[0] - class1_cov))
eucl_dist2 = np.sum(np.abs(covs_train[0] - class2_cov))

print(f'Euclidean distance between an example and class 0: {eucl_dist0:0.2f}')
print(f'Euclidean distance between an example and class 1: {eucl_dist1:0.2f}')
print(f'Euclidean distance between an example and class 2: {eucl_dist2:0.2f}')

rd0 = distance_riemann(covs_train[0], class0_cov)
rd1 = distance_riemann(covs_train[0], class1_cov)
rd2 = distance_riemann(covs_train[0], class2_cov)

print(f'Geometric distance between an example and class 0: {rd0:0.2f}')
print(f'Geometric distance between an example and class 1: {rd1:0.2f}')
print(f'Geometric distance between an example and class 2: {rd2:0.2f}')

Euclidean distance between an example and class 0: 0.00
Euclidean distance between an example and class 1: 0.00
Euclidean distance between an example and class 2: 0.00
Geometric distance between an example and class 0: 81.83
Geometric distance between an example and class 1: 83.96
Geometric distance between an example and class 2: 84.28


In [6]:
# Cal simple np covariance mats

cov1 = Covariances('oas').fit_transform(X1)
cov2 = Covariances('oas').fit_transform(X2)

In [8]:
# Simple MDM based classifier

mdmc = MDM(n_jobs=-1)
mdmc.fit(cov1,y1)
# test_acc = mdmc.score(cov2, y2)
pm = mdmc.predict_proba(cov2)
print(pm.shape)
# print(y2)

# print(f'\nAccuracy of MDM Classifier: {test_acc * 100:0.2f}%\n')

# mdmc2 = MDM(n_jobs=-1)
# mdmc2.fit(cov2,y2)
# test_acc2 = mdmc2.score(cov1, y1)

# print(f'\nAccuracy of MDM Classifier 2: {test_acc2 * 100:0.2f}%\n')

(447, 3)


In [8]:

Fgmdmc = FgMDM(n_jobs=-1)
Fgmdmc.fit(cov1,y1)
# test_acc = Fgmdmc.score(cov2, y2)

# print(f'\nAccuracy of FgMDM Classifier: {test_acc * 100:0.2f}%\n')
pfgm = 

Fgmdmc2 = FgMDM(n_jobs=-1)
Fgmdmc2.fit(cov2,y2)
test_acc2 = Fgmdmc2.score(cov1, y1)

print(f'\nAccuracy of FgMDM Classifier 2: {test_acc2 * 100:0.2f}%\n')


Accuracy of FgMDM Classifier: 35.57%


Accuracy of FgMDM Classifier 2: 33.33%



In [31]:
Fgmdmc3 = FgMDM(n_jobs=-1,tsupdate=True)
Fgmdmc3.fit(cov2,y2)
# test_acc2 = Fgmdmc2.score(cov1, y1)
print("training score ",Fgmdmc3.score(cov2,y2))
print("test score ",Fgmdmc3.score(cov1,y1))

training score  1.0
test score  0.2751677852348993


In [9]:
# Simply put, this projects to TS and applies LogReg
tsc1 = tsc()
tsc1.fit(cov1,y1)
test_acc = tsc1.score(cov2, y2)

print(f'\nAccuracy of tsc Classifier: {test_acc * 100:0.2f}%\n')

tsc2 = tsc()
tsc2.fit(cov2,y2)
test_acc2 = tsc2.score(cov1, y1)

print(f'\nAccuracy of tsc Classifier 2: {test_acc2 * 100:0.2f}%\n')


Accuracy of tsc Classifier: 36.02%


Accuracy of tsc Classifier 2: 40.27%



In [10]:
# This enables covariate shift based on test data.. Not applicable for online eval
tsc1 = tsc(tsupdate=True)
tsc1.fit(cov1,y1)
test_acc = tsc1.score(cov2, y2)

print(f'\nAccuracy of tsc Classifier: {test_acc * 100:0.2f}%\n')

tsc2 = tsc(tsupdate=True)
tsc2.fit(cov2,y2)
test_acc2 = tsc2.score(cov1, y1)

print(f'\nAccuracy of tsc Classifier 2: {test_acc2 * 100:0.2f}%\n')


Accuracy of tsc Classifier: 45.41%


Accuracy of tsc Classifier 2: 52.57%



In [11]:

knn1 = knn(n_jobs=-1)
knn1.fit(cov1,y1)
test_acc = knn1.score(cov2, y2)

print(f'\nAccuracy of knn Classifier: {test_acc * 100:0.2f}%\n')

knn2 = knn(n_jobs=-1)
knn2.fit(cov2,y2)
test_acc2 = knn2.score(cov1, y1)

print(f'\nAccuracy of knn Classifier 2: {test_acc2 * 100:0.2f}%\n')

# knn pyriemann doesn't have predict_proba for itself 


Accuracy of knn Classifier: 39.15%


Accuracy of knn Classifier 2: 54.14%



In [44]:
knn2 = knn(n_jobs=-1)
knn2.fit(cov2,y2)
p = knn2.predict_proba(cov1)
p.shape

(447, 447)

In [47]:
cov2.shape

(447, 61, 61)

In [46]:
# y2
knn3 = sknn(n_jobs=-1,metric = distance_riemann)
knn3.fit(cov2,y2)
print("sk knn train = ",knn3.score(cov2,y2)," test = ",knn3.score(X1,y1))

In [14]:
# Now do a voting classifier based on these 4/5 (depending on tsupdate as second)


mdmc = MDM(n_jobs=-1)
Fgmdmc = FgMDM(n_jobs=-1)
tsc1 = tsc(tsupdate=True)
knn1 = knn(n_jobs=-1)

svoc = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc1)], voting='soft',n_jobs=-1)
svoc.fit(cov1,y1)
# print(svoc.predict(cov1))

VotingClassifier(estimators=[('mdm', MDM(n_jobs=-1)),
                             ('Fgmdm', FgMDM(n_jobs=-1)),
                             ('tsc', TSclassifier(tsupdate=True))],
                 n_jobs=-1, voting='soft')

In [16]:
print("training score ",svoc.score(cov1,y1))

print("test score ",svoc.score(cov2,y2))

training score  1.0
test score  0.4519015659955257


In [ ]:
mdmc = MDM(n_jobs=-1)
Fgmdmc = FgMDM(n_jobs=-1)
tsc1 = tsc(tsupdate=True)
knn1 = knn(n_jobs=-1)

svoc = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc1)], voting='soft',n_jobs=-1)
svoc.fit(cov1,y1)

In [21]:
svoc2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc1)], voting='soft',n_jobs=-1,flatten_transform = False)
svoc2.fit(cov2,y2)

VotingClassifier(estimators=[('mdm', MDM(n_jobs=-1)),
                             ('Fgmdm', FgMDM(n_jobs=-1)),
                             ('tsc', TSclassifier(tsupdate=True))],
                 n_jobs=-1, voting='soft')

In [22]:
print("training score ",svoc2.score(cov2,y2))

print("test score ",svoc2.score(cov1,y1))

training score  1.0
test score  0.5480984340044742


In [24]:
import seaborn as sns

# View the individual prediction to see if I can possibly weight some classifier accordingly
p1 = svoc2.transform(cov1)
p2 = svoc2.transform(cov2) #trained on

%matplotlib qt
sns.heatmap(p1)
plt.show()

In [25]:
p1.shape

(447, 9)

In [26]:
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc1)], voting='hard',n_jobs=-1)
svoc3.fit(cov2,y2)

VotingClassifier(estimators=[('mdm', MDM(n_jobs=-1)),
                             ('Fgmdm', FgMDM(n_jobs=-1)),
                             ('tsc', TSclassifier(tsupdate=True))],
                 n_jobs=-1)

In [27]:
print("training score ",svoc3.score(cov2,y2))

print("test score ",svoc3.score(cov1,y1))

training score  1.0
test score  0.5816554809843401


In [28]:
tsc3 = tsc()
svoc4 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc3)], voting='hard',n_jobs=-1)
svoc4.fit(cov2,y2)

VotingClassifier(estimators=[('mdm', MDM(n_jobs=-1)),
                             ('Fgmdm', FgMDM(n_jobs=-1)),
                             ('tsc', TSclassifier())],
                 n_jobs=-1)

In [29]:
print("training score ",svoc4.score(cov2,y2))
print("test score ",svoc4.score(cov1,y1))

training score  1.0
test score  0.4272930648769575


In [50]:
mdmc = make_pipeline(Covariances(estimator='oas'),MDM(n_jobs=-1))
Fgmdmc = make_pipeline(Covariances(estimator='oas'),FgMDM(n_jobs=-1))
tsc1 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True))
knn1 = make_pipeline(Covariances(estimator='oas'),knn(n_jobs=-1))

svoc = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc1)], voting='hard',n_jobs=-1)
svoc.fit(X2,y2)

print("svoc hard train = ",svoc.score(X2,y2)," test = ",svoc.score(X1,y1))

svoc hard train =  1.0  test =  0.5816554809843401


In [60]:
svoc2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc1)], voting='hard',n_jobs=-1)
svoc2.fit(X1,y1)

print("svoc2 hard train = ",svoc2.score(X1,y1)," test = ",svoc2.score(X2,y2))

svoc2 hard train =  1.0  test =  0.4317673378076063


In [58]:
tsc2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_jobs=-1)))
tsc2.fit(X2,y2)
print("tsc-sknn train = ",tsc2.score(X2,y2)," test = ",tsc2.score(X1,y1))

tsc-sknn train =  0.8926174496644296  test =  0.5167785234899329


In [63]:
tsc3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_jobs=-1)))
tsc3.fit(X2,y2)
p2 = tsc3.predict(X2)
p1 = tsc3.predict(X1)

# print("tsc-sknn train = ",tsc3.score(X2,y2)," test = ",tsc3.score(X1,y1))
print("train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

D:\Softwares\Anaconda3\envs\neuroerg\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[00:28:06] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
train =  1.0  test =  0.5167785234899329


In [67]:
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc3)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)

print("svoc3 hard train = ",svoc3.score(X2,y2)," test = ",svoc3.score(X1,y1))

svoc3 hard train =  1.0  test =  0.5906040268456376


In [108]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1000,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svoc3 hard train =  1.0  test =  0.5973154362416108


In [109]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1000,max_depth = 10,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svoc3 hard train =  1.0  test =  0.5995525727069351


In [110]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1000,max_depth = 10,learning_rate= 0.6,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svoc3 hard train =  1.0  test =  0.6129753914988815


In [118]:


cx = np.around(cm(y1,p1,normalize='true'),2)
cx

array([[0.59, 0.17, 0.23],
       [0.05, 0.78, 0.17],
       [0.13, 0.4 , 0.47]])

In [124]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.8,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cx = np.around(cm(y1,p1,normalize='true'),2)
cx

svoc3 hard train =  1.0  test =  0.5883668903803132


array([[0.54, 0.13, 0.33],
       [0.07, 0.76, 0.17],
       [0.15, 0.38, 0.46]])

In [125]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.5,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cx = np.around(cm(y1,p1,normalize='true'),2)
cx

svoc3 hard train =  1.0  test =  0.6152125279642058


array([[0.62, 0.13, 0.24],
       [0.07, 0.76, 0.17],
       [0.17, 0.36, 0.46]])

In [128]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cx = np.around(cm(y1,p1,normalize='true'),2)
cx

svoc3 hard train =  1.0  test =  0.6174496644295302


array([[0.61, 0.14, 0.25],
       [0.07, 0.77, 0.16],
       [0.15, 0.38, 0.47]])

In [127]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cx = np.around(cm(y1,p1,normalize='true'),2)
cx

svoc3 hard train =  1.0  test =  0.6442953020134228


array([[0.67, 0.12, 0.21],
       [0.05, 0.77, 0.17],
       [0.13, 0.38, 0.49]])

In [129]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.1,reg_lambda = 5,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cx = np.around(cm(y1,p1,normalize='true'),2)
cx

svoc3 hard train =  1.0  test =  0.6420581655480985


array([[0.68, 0.15, 0.17],
       [0.06, 0.76, 0.18],
       [0.13, 0.38, 0.49]])

In [130]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,gamma = 0.3,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cx = np.around(cm(y1,p1,normalize='true'),2)
cx

svoc3 hard train =  1.0  test =  0.5928411633109619


array([[0.56, 0.13, 0.3 ],
       [0.07, 0.76, 0.17],
       [0.14, 0.4 , 0.46]])

In [131]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,colsample_bytree = 0.7,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cx = np.around(cm(y1,p1,normalize='true'),2)
cx

svoc3 hard train =  1.0  test =  0.6085011185682326


array([[0.61, 0.07, 0.32],
       [0.08, 0.72, 0.2 ],
       [0.13, 0.37, 0.5 ]])

In [70]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cx = np.around(cm(y1,p1,normalize='true'),2)
cx

In [70]:
tsc2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc2)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svoc3 hard train =  0.9284116331096197  test =  0.6174496644295302


In [73]:
p1

array([2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 2, 2, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 2, 1, 0, 2, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 2, 0, 0, 1, 1, 0, 0, 2,

In [74]:
tsck1 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 10,n_jobs=-1)))
svoc4 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsck1)], voting='hard',n_jobs=-1)
svoc4.fit(X2,y2)
p1 = svoc4.predict(X1)
p2 = svoc4.predict(X2)
print("svoc4 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svoc4 hard train =  0.9038031319910514  test =  0.6465324384787472


In [79]:
tsck2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 10,n_jobs=-1,weights='distance')))
svock2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsck2)], voting='hard',n_jobs=-1)
svock2.fit(X2,y2)
p1 = svock2.predict(X1)
p2 = svock2.predict(X2)
print("svock2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svock2 hard train =  1.0  test =  0.6532438478747203


In [80]:
tsck2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 22,n_jobs=-1,weights='distance')))
svock2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsck2)], voting='hard',n_jobs=-1)
svock2.fit(X2,y2)
p1 = svock2.predict(X1)
p2 = svock2.predict(X2)
print("svock2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svock2 hard train =  1.0  test =  0.6420581655480985


In [84]:
tsck2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 14,n_jobs=-1,weights='distance')))
svock2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsck2)], voting='hard',n_jobs=-1)
svock2.fit(X2,y2)
p1 = svock2.predict(X1)
p2 = svock2.predict(X2)
print("svock2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svock2 hard train =  1.0  test =  0.6420581655480985


In [86]:
p1

array([2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 2, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 2, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 2, 1, 1, 1, 2, 1, 1, 1, 1, 0, 0, 2,

In [77]:
# tsck4 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 14,n_jobs=-1,weights='distance',metric=)))
# svock4 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsck4)], voting='hard',n_jobs=-1)
# svock4.fit(X2,y2)
# p1 = svock4.predict(X1)
# p2 = svock4.predict(X2)
# print("svock4 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

In [77]:

tscs1 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC()))
svoc5 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs1)], voting='hard',n_jobs=-1)
svoc5.fit(X2,y2)
p1 = svoc5.predict(X1)
p2 = svoc5.predict(X2)
print("svoc5 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svoc5 hard train =  1.0  test =  0.6353467561521253


In [92]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(C=10)))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
p1 = svocs2.predict(X1)
p2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svocs2 hard train =  1.0  test =  0.6241610738255033


In [93]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(C=1000)))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
p1 = svocs2.predict(X1)
p2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svocs2 hard train =  1.0  test =  0.6241610738255033


In [103]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
p1 = svocs2.predict(X1)
p2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svocs2 hard train =  0.901565995525727  test =  0.7002237136465325


In [104]:
p1

array([2, 2, 0, 2, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 2, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       2, 0, 0, 2, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 2, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 2, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 2, 1, 0, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,

In [98]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto',C=10)))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
p1 = svocs2.predict(X1)
p2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svocs2 hard train =  0.9955257270693513  test =  0.5682326621923938


In [101]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto',C=0.1)))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
p1 = svocs2.predict(X1)
p2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svocs2 hard train =  0.8657718120805369  test =  0.6666666666666666


In [102]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto',C=0.5)))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
p1 = svocs2.predict(X1)
p2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svocs2 hard train =  0.8747203579418344  test =  0.6756152125279642


In [99]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto',C=100)))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
p1 = svocs2.predict(X1)
p2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svocs2 hard train =  1.0  test =  0.5637583892617449


In [100]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto',C=1000)))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
p1 = svocs2.predict(X1)
p2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svocs2 hard train =  1.0  test =  0.5637583892617449


In [95]:
tscs3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC()))
svocs3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs3)], voting='hard',n_jobs=-1)
svocs3.fit(X2,y2)
p1 = svocs3.predict(X1)
p2 = svocs3.predict(X2)
print("svocs3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

svocs3 hard train =  1.0  test =  0.6756152125279642


In [97]:
tscs3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC(gamma='auto',C=100)))
svocs3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs3)], voting='hard',n_jobs=-1)
svocs3.fit(X2,y2)
p1 = svocs3.predict(X1)
p2 = svocs3.predict(X2)
print("svocs3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

TypeError: __init__() got an unexpected keyword argument 'C'

In [52]:
from sklearn.svm import NuSVC
tscs3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC(C=100)))
svocs3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs3)], voting='hard',n_jobs=-1)
svocs3.fit(X2,y2)
p1 = svocs3.predict(X1)
p2 = svocs3.predict(X2)
print("svocs3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

In [52]:
#stacking classifier

sc1 = SC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc1)], cv = 5,n_jobs=-1)
sc1.fit(X2,y2)

print("stackclass test = ",sc1.score(X1,y1), " train = ",sc1.score(X2,y2))

stackclass test =  0.4742729306487696  train =  1.0


In [53]:
sc2 = SC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc1)], cv = 5,n_jobs=-1,passthrough=True)
sc2.fit(X2,y2)

print("stackclass test = ",sc2.score(X1,y1), " train = ",sc2.score(X2,y2))

ValueError: all the input arrays must have same number of dimensions, but the array at index 0 has 2 dimension(s) and the array at index 3 has 3 dimension(s)

In [55]:
sc3 = SC(estimators = [('knn',knn1),('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsc1)], stack_method = 'predict', cv = 5,n_jobs=-1)
sc3.fit(X2,y2)

print("stackclass test = ",sc3.score(X1,y1), " train = ",sc3.score(X2,y2))

stackclass test =  0.2841163310961969  train =  1.0


In [160]:
ina2 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
                    TangentSpace(),
                    sknn(n_neighbors = 14,n_jobs=-1,weights='distance'))
ina2.fit(X2,y2)
p1 = ina2.predict(X1)
p2 = ina2.predict(X2)
print("tsc-svc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

tsc-svc hard train =  1.0  test =  0.5100671140939598


array([[0.19, 0.62, 0.19],
       [0.01, 0.71, 0.28],
       [0.01, 0.37, 0.62]])

In [161]:
ina2 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
                    TangentSpace(metric='logeuclid'),
                    xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1))
ina2.fit(X2,y2)
p1 = ina2.predict(X1)
p2 = ina2.predict(X2)
print("tsc-svc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

D:\Softwares\Anaconda3\envs\neuroerg\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:49:48] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
tsc-svc hard train =  1.0  test =  0.4317673378076063


array([[0.43, 0.48, 0.09],
       [0.13, 0.87, 0.01],
       [0.37, 0.63, 0.  ]])

In [162]:
ina2 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
                    TangentSpace(metric='logeuclid'),
                    NuSVC())
ina2.fit(X2,y2)
p1 = ina2.predict(X1)
p2 = ina2.predict(X2)
print("tsc-svc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

tsc-svc hard train =  1.0  test =  0.5727069351230425


array([[0.52, 0.15, 0.33],
       [0.11, 0.25, 0.64],
       [0.03, 0.01, 0.95]])

In [156]:
ina2 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
                    TangentSpace(metric='logeuclid'),
                    sknn(n_neighbors = 14,n_jobs=-1,weights='distance'))
ina2.fit(X2,y2)
p1 = ina2.predict(X1)
p2 = ina2.predict(X2)
print("tsc-svc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

tsc-svc hard train =  1.0  test =  0.5078299776286354


array([[0.19, 0.72, 0.09],
       [0.02, 0.72, 0.26],
       [0.01, 0.39, 0.6 ]])

In [157]:
ina3 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
                    CSP(30),
                    sknn(n_neighbors = 14,n_jobs=-1,weights='distance'))
ina3.fit(X2,y2)
p1 = ina3.predict(X1)
p2 = ina3.predict(X2)
print("tsc-svc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

tsc-svc hard train =  1.0  test =  0.39373601789709173


array([[0.  , 0.  , 1.  ],
       [0.04, 0.19, 0.77],
       [0.  , 0.01, 0.99]])

In [158]:
ina3 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
                    CSP(30),
                    xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1))
ina3.fit(X2,y2)
p1 = ina3.predict(X1)
p2 = ina3.predict(X2)
print("tsc-svc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

D:\Softwares\Anaconda3\envs\neuroerg\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:35:36] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
tsc-svc hard train =  1.0  test =  0.3959731543624161


array([[0.  , 0.02, 0.98],
       [0.06, 0.21, 0.72],
       [0.  , 0.03, 0.97]])

In [159]:
ina3 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
                    CSP(30),
                    NuSVC())
ina3.fit(X2,y2)
p1 = ina3.predict(X1)
p2 = ina3.predict(X2)
print("tsc-svc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c


tsc-svc hard train =  0.959731543624161  test =  0.3959731543624161


array([[0.  , 0.01, 0.99],
       [0.01, 0.19, 0.8 ],
       [0.  , 0.  , 1.  ]])

In [34]:
# erp
erp1 = make_pipeline(XdawnCovariances(6, estimator='oas'),
                    TangentSpace(metric='riemann',tsupdate=True),
                    LogisticRegression(penalty='l2'))
erp2 = make_pipeline(Xdawn(12, estimator='oas'),
                    EpochsVectorizer(),
                    LogisticRegression(penalty='l2'))

In [39]:
# induced activity
# removed electrode selection from the base classifier
baseclf = make_pipeline(TangentSpace(metric='riemann', tsupdate=True),
                        LogisticRegression(penalty='l1',solver='liblinear'))
ina1 = make_pipeline(CospCovariances(fs=250, window=125, overlap=0, fmax=45, fmin=0.1),
                    CospBoostingClassifier(baseclf))

In [38]:
ina2 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
                    TangentSpace(metric='logeuclid'),
                    LogisticRegression(penalty='l1',solver='liblinear'))

ina3 = make_pipeline(HankelCovariances(delays=[2, 4, 8, 12, 16], estimator='oas'),
                    CSP(30),
                    LogisticRegression(penalty='l1',solver='liblinear'))

In [42]:
# ,("ina1",ina1)
eclf1 = VC(estimators = [("erp1",erp1),("erp2",erp2),("ina2",ina2),("ina3",ina3)],
                         voting='hard',n_jobs=-1)

eclf2 = VC(estimators = [("erp1",erp1),("erp2",erp2),("ina2",ina2),("ina3",ina3)],
                         voting='soft',n_jobs=-1)

In [35]:
erp1.fit(X2,y2)
print("erp1 train = ",erp1.score(X2,y2)," test = ",erp1.score(X1,y1))
erp2.fit(X2,y2)
print("erp2 train = ",erp2.score(X2,y2)," test = ",erp2.score(X1,y1))

In [35]:
ina1.fit(X2,y2)
print("ina1 train = ",ina1.score(X2,y2)," test = ",ina1.score(X1,y1))

erp1 train =  1.0  test =  0.31543624161073824
erp2 train =  0.3333333333333333  test =  0.3333333333333333


D:\Softwares\Anaconda3\envs\neuroerg\lib\site-packages\pyriemann\utils\base.py:14: RuntimeWarning: invalid value encountered in log
  eigvals = np.diag(operator(eigvals))


ValueError: Covariance matrices must be positive definite. Add regularization to avoid this error.

In [41]:
ina2.fit(X2,y2)
print("ina2 train = ",ina2.score(X2,y2)," test = ",ina2.score(X1,y1))
ina3.fit(X2,y2)
print("ina3 train = ",ina3.score(X2,y2)," test = ",ina3.score(X1,y1))

ina2 train =  0.9977628635346756  test =  0.30648769574944074
ina3 train =  0.9686800894854586  test =  0.38926174496644295


In [43]:
eclf1.fit(X2,y2)
print("eclf1 train = ",eclf1.score(X2,y2)," test = ",eclf1.score(X1,y1))
eclf2.fit(X2,y2)
print("eclf2 train = ",eclf2.score(X2,y2)," test = ",eclf2.score(X1,y1))

eclf1 train =  0.9821029082774049  test =  0.2595078299776286
eclf2 train =  1.0  test =  0.38926174496644295


In [18]:
#XdawnCovariances - for better generalization capability set applyfilters=False

In [18]:
# %run funcs.ipynb
cov1.shape
# y1.shape

(447, 61, 61)

In [48]:
# can I get the class covariances for the 2 sessions to see if they are similar or different?

In [48]:
# print(cov2.shape)

(447, 61, 61)


In [36]:
# Misc
# covs_train[0].shape
# y1
# class1_cov.shape

# a = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
# len(a)

# y1.shape

In [37]:
# Best of knn, svc and gradboost

In [147]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
tscs2.fit(X2,y2)
p1 = tscs2.predict(X1)
p2 = tscs2.predict(X2)
print("tsc-svc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

tsc-svc hard train =  0.8903803131991052  test =  0.6241610738255033


array([[0.95, 0.03, 0.01],
       [0.36, 0.6 , 0.05],
       [0.25, 0.43, 0.32]])

In [107]:
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
svocs2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2)], voting='hard',n_jobs=-1)
svocs2.fit(X2,y2)
ps1 = svocs2.predict(X1)
ps2 = svocs2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,ps2)," test = ",accuracy_score(y1,ps1))

svocs2 hard train =  0.901565995525727  test =  0.7002237136465325


In [120]:
csx = np.around(cm(y1,ps1,normalize='true'),2)
csx

array([[0.88, 0.03, 0.09],
       [0.05, 0.9 , 0.05],
       [0.12, 0.56, 0.32]])

In [123]:
csxt = np.around(cm(y1,ps2,normalize='true'),2)
csxt

array([[0.91, 0.08, 0.01],
       [0.19, 0.8 , 0.01],
       [0.  , 0.  , 1.  ]])

In [148]:
tscs3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC()))
tscs3.fit(X2,y2)
p1 = tscs3.predict(X1)
p2 = tscs3.predict(X2)
print("tsc-nusvc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

tsc-nusvc hard train =  1.0  test =  0.6241610738255033


array([[0.91, 0.07, 0.01],
       [0.29, 0.65, 0.06],
       [0.31, 0.38, 0.31]])

In [105]:
tscs3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC()))
svocs3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs3)], voting='hard',n_jobs=-1)
svocs3.fit(X2,y2)
pn1 = svocs3.predict(X1)
pn2 = svocs3.predict(X2)
print("svocs3 hard train = ",accuracy_score(y2,pn2)," test = ",accuracy_score(y1,pn1))

svocs3 hard train =  1.0  test =  0.6756152125279642


In [119]:
cnx = np.around(cm(y1,pn1,normalize='true'),2)
cnx

array([[0.84, 0.07, 0.09],
       [0.06, 0.88, 0.06],
       [0.16, 0.53, 0.31]])

In [150]:
tsck2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 14,n_jobs=-1,weights='distance')))
tsck2.fit(X2,y2)
p1 = tsck2.predict(X1)
p2 = tsck2.predict(X2)
print("tsc-skknn hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

tsc-skknn hard train =  1.0  test =  0.5794183445190156


array([[0.95, 0.03, 0.02],
       [0.36, 0.54, 0.11],
       [0.39, 0.36, 0.25]])

In [121]:
tsck2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 14,n_jobs=-1,weights='distance')))
svock2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tsck2)], voting='hard',n_jobs=-1)
svock2.fit(X2,y2)
pk1 = svock2.predict(X1)
pk2 = svock2.predict(X2)
print("svock2 hard train = ",accuracy_score(y2,pk2)," test = ",accuracy_score(y1,pk1))

svock2 hard train =  1.0  test =  0.6554809843400448


In [122]:
ckx = np.around(cm(y1,pk1,normalize='true'),2)
ckx

array([[0.87, 0.03, 0.1 ],
       [0.05, 0.85, 0.11],
       [0.19, 0.56, 0.25]])

In [149]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
tscx.fit(X2,y2)
p1 = tscx.predict(X1)
p2 = tscx.predict(X2)
print("tscx-nusvc hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

c = np.around(cm(y1,p1,normalize='true'),2)
c

D:\Softwares\Anaconda3\envs\neuroerg\lib\site-packages\xgboost\sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[04:43:56] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.4.0/src/learner.cc:1095: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
tscx-nusvc hard train =  1.0  test =  0.5883668903803132


array([[0.66, 0.18, 0.15],
       [0.23, 0.47, 0.3 ],
       [0.22, 0.15, 0.63]])

In [132]:
# xgb gives much better result for 3rd class
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
svoc3 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscx)], voting='hard',n_jobs=-1)
svoc3.fit(X2,y2)
p1 = svoc3.predict(X1)
p2 = svoc3.predict(X2)
print("svoc3 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cx = np.around(cm(y1,p1,normalize='true'),2)
cx

svoc3 hard train =  1.0  test =  0.6442953020134228


array([[0.67, 0.12, 0.21],
       [0.05, 0.77, 0.17],
       [0.13, 0.38, 0.49]])

In [133]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
svocsx2 = VC(estimators = [('mdm',mdmc),('Fgmdm',Fgmdmc),('tsc',tscs2),('tscx',tscx)], voting='hard',n_jobs=-1)
svocsx2.fit(X2,y2)
psx1 = svocsx2.predict(X1)
psx2 = svocsx2.predict(X2)
print("svocs2 hard train = ",accuracy_score(y2,psx2)," test = ",accuracy_score(y1,psx1))

svocs2 hard train =  0.9373601789709173  test =  0.639821029082774


In [134]:
csx = np.around(cm(y1,psx1,normalize='true'),2)
csx

array([[0.77, 0.13, 0.11],
       [0.16, 0.79, 0.05],
       [0.11, 0.53, 0.36]])

In [151]:
ms = VC(estimators = [('mdm',mdmc),('tscx',tscx)],voting='hard',n_jobs=-1)
ms.fit(X2,y2)
p1 = ms.predict(X1)
p2 = ms.predict(X2)
print("ms hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

cf = np.around(cm(y1,p1,normalize='true'),2)
cf

ms hard train =  0.9105145413870246  test =  0.6174496644295302


array([[0.77, 0.19, 0.03],
       [0.23, 0.59, 0.17],
       [0.24, 0.27, 0.49]])

In [152]:
ms = VC(estimators = [('mdm',mdmc),('tscx',tscx)],voting='soft',n_jobs=-1)
ms.fit(X2,y2)
p1 = ms.predict(X1)
p2 = ms.predict(X2)
print("ms hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

cf = np.around(cm(y1,p1,normalize='true'),2)
cf

ms hard train =  0.9485458612975392  test =  0.5995525727069351


array([[0.46, 0.41, 0.13],
       [0.05, 0.57, 0.38],
       [0.06, 0.17, 0.77]])

In [153]:
ms = VC(estimators = [('mdm',mdmc),('tscx',tscx),('NuSVC',tscs3)],voting='hard',n_jobs=-1)
ms.fit(X2,y2)
p1 = ms.predict(X1)
p2 = ms.predict(X2)
print("ms hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

cf = np.around(cm(y1,p1,normalize='true'),2)
cf

ms hard train =  1.0  test =  0.6510067114093959


array([[0.83, 0.14, 0.03],
       [0.22, 0.57, 0.21],
       [0.23, 0.21, 0.56]])

In [155]:
tscs3 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=NuSVC(probability=True)))
ms = VC(estimators = [('mdm',mdmc),('tscx',tscx),('NuSVC',tscs3)],voting='soft',n_jobs=-1)
ms.fit(X2,y2)
p1 = ms.predict(X1)
p2 = ms.predict(X2)
print("ms hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))

cf = np.around(cm(y1,p1,normalize='true'),2)
cf

ms hard train =  1.0  test =  0.6666666666666666


array([[0.7 , 0.23, 0.07],
       [0.09, 0.67, 0.24],
       [0.11, 0.26, 0.63]])

In [135]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
svocnsx2 = VC(estimators = [('tsc',tscs2),('tscx',tscx)], voting='hard',n_jobs=-1)
svocnsx2.fit(X2,y2)
pnsx1 = svocnsx2.predict(X1)
pnsx2 = svocnsx2.predict(X2)
print("svocnsx2 hard train = ",accuracy_score(y2,pnsx2)," test = ",accuracy_score(y1,pnsx1))
csx = np.around(cm(y1,pnsx1,normalize='true'),2)
csx

svocnsx2 hard train =  0.9261744966442953  test =  0.5861297539149888


array([[0.96, 0.03, 0.01],
       [0.43, 0.53, 0.04],
       [0.38, 0.35, 0.27]])

In [142]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
mdmc = make_pipeline(Covariances(estimator='oas'),MDM(n_jobs=-1))
sf1 = VC(estimators = [('tsc',tscs2),('tscx',tscx),('mdm',mdmc)],weights = [1,1,2], voting='hard',n_jobs=-1)
sf1.fit(X2,y2)
p1 = sf1.predict(X1)
p2 = sf1.predict(X2)
print("sf1 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cf = np.around(cm(y1,p1,normalize='true'),2)
cf

sf1 hard train =  0.8747203579418344  test =  0.6599552572706935


array([[0.77, 0.19, 0.04],
       [0.16, 0.62, 0.21],
       [0.11, 0.3 , 0.59]])

In [144]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto')))
mdmc = make_pipeline(Covariances(estimator='oas'),MDM(n_jobs=-1))
tsck2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 14,n_jobs=-1,weights='distance')))

sf1 = VC(estimators = [('tsc',tscs2),('tscx',tscx),('tskc',tsck2),('mdm',mdmc)],weights = [1,1,1,4], voting='hard',n_jobs=-1)
sf1.fit(X2,y2)
p1 = sf1.predict(X1)
p2 = sf1.predict(X2)
print("sf1 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cf = np.around(cm(y1,p1,normalize='true'),2)
cf

sf1 hard train =  0.843400447427293  test =  0.5257270693512305


array([[0.28, 0.63, 0.09],
       [0.03, 0.6 , 0.38],
       [0.03, 0.27, 0.7 ]])

In [146]:
tscx = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=xgb.XGBClassifier(n_estimators = 1500,max_depth = 15,learning_rate= 0.3,reg_lambda = 5,n_jobs=-1)))
tscs2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=SVC(gamma='auto',probability=True)))
mdmc = make_pipeline(Covariances(estimator='oas'),MDM(n_jobs=-1))
tsck2 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True,clf=sknn(n_neighbors = 14,n_jobs=-1,weights='distance')))

sf1 = VC(estimators = [('tsc',tscs2),('tscx',tscx),('tskc',tsck2),('mdm',mdmc)],weights = [1,1,1,4], voting='soft',n_jobs=-1)
sf1.fit(X2,y2)
p1 = sf1.predict(X1)
p2 = sf1.predict(X2)
print("sf1 hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
cf = np.around(cm(y1,p1,normalize='true'),2)
cf 

sf1 hard train =  0.8814317673378076  test =  0.5749440715883669


array([[0.42, 0.49, 0.09],
       [0.03, 0.62, 0.34],
       [0.05, 0.27, 0.68]])

In [137]:
mdmc = make_pipeline(Covariances(estimator='oas'),MDM(n_jobs=-1))
mdmc.fit(X2,y2)
p1 = mdmc.predict(X1)
p2 = mdmc.predict(X2)
print("indiv hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
c = np.around(cm(y1,p1,normalize='true'),2)
c

indiv hard train =  0.843400447427293  test =  0.5257270693512305


array([[0.28, 0.63, 0.09],
       [0.03, 0.6 , 0.38],
       [0.03, 0.27, 0.7 ]])

In [138]:
Fgmdmc = make_pipeline(Covariances(estimator='oas'),FgMDM(n_jobs=-1))
Fgmdmc.fit(X2,y2)
p1 = Fgmdmc.predict(X1)
p2 = Fgmdmc.predict(X2)
print("indiv hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
c = np.around(cm(y1,p1,normalize='true'),2)
c

indiv hard train =  1.0  test =  0.3333333333333333


array([[0.  , 0.01, 0.99],
       [0.  , 1.  , 0.  ],
       [0.01, 0.99, 0.  ]])

In [139]:
Fgmdmc = make_pipeline(Covariances(estimator='oas'),FgMDM(tsupdate=True,n_jobs=-1))
Fgmdmc.fit(X2,y2)
p1 = Fgmdmc.predict(X1)
p2 = Fgmdmc.predict(X2)
print("indiv hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
c = np.around(cm(y1,p1,normalize='true'),2)
c

indiv hard train =  1.0  test =  0.2751677852348993


array([[0.25, 0.08, 0.67],
       [0.42, 0.58, 0.  ],
       [0.5 , 0.5 , 0.  ]])

In [140]:
tsc1 = make_pipeline(Covariances(estimator='oas'),tsc(tsupdate=True))
tsc1.fit(X2,y2)
p1 = tsc1.predict(X1)
p2 = tsc1.predict(X2)
print("indiv hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
c = np.around(cm(y1,p1,normalize='true'),2)
c

indiv hard train =  1.0  test =  0.5257270693512305


array([[0.68, 0.14, 0.17],
       [0.26, 0.72, 0.01],
       [0.36, 0.47, 0.17]])

In [141]:
knn1 = make_pipeline(Covariances(estimator='oas'),knn(n_jobs=-1))
knn1.fit(X2,y2)
p1 = knn1.predict(X1)
p2 = knn1.predict(X2)
print("indiv hard train = ",accuracy_score(y2,p2)," test = ",accuracy_score(y1,p1))
c = np.around(cm(y1,p1,normalize='true'),2)
c

indiv hard train =  0.9038031319910514  test =  0.5413870246085011


array([[0.79, 0.17, 0.05],
       [0.26, 0.68, 0.07],
       [0.35, 0.49, 0.16]])

In [ ]:
# Stacking classifier
sc1 = SC(estimators = [])